# 70019 Probabilistic Inference  
Assessed Coursework #1 (of 1)

Coursework to be done either individually or in groups of two individuals

## Learning goals

This coursework helps you practice:

• (P1) implementing Bayesian inference efficiently by exploiting model structure,  
• (P1) understanding why this can be dramatically faster than naive marginalisation,  
• (P2) understanding Gaussian processes as distributions over functions.  
• (P2) implementing GP regression from scratch using NumPy.

## Rules / Allowed tools

• Use NumPy only for calculations (and optionally matplotlib for plotting).  
• Do not use HMM/GP/probabilistic inference libraries (e.g., hmmlearn, pgmpy, pyro, etc.).  
• You may write helper functions, add tests, and refactor code as you like.  
• Please include an LLM Statement at the end noting if/how you used LLMs.

## Problem 1

As an instructor, you want to understand a student's workload level over time, but you cannot observe/experience this directly (workload is a hidden state). Each day, you observe the student's engagement in your class, which is a noisy indicator of workload. Mathematically speaking, for each time step $t = 1, . . . , T$, we define the hidden state

$$
X_t \in \{0, 1, 2, 3, 4\}
$$

where 0 means very low workload and 4 means extreme workload. At $t = 1, . . . , T$, you observe:

$$
y_{1:6} = [\text{high, medium, low, absent, low, medium}].
$$

Use this provided example sequence for debugging and comparison. For timing, you may create a longer sequence of length $T = 50$ by repeating a pattern (or sampling) of engagement symbols.

## Model assumptions (HMM)

You assume a hidden Markov model:

$$
p(x_1) = \pi(x_1), \quad p(x_t \mid x_{t-1}) = A[x_{t-1}, x_t], \quad p(y_t \mid x_t) = B[x_t, y_t].
$$

with hidden state size $|X| = 5$ and observation size $|Y| = 4$ as above.

---

## Model parameters

You assume the initial distribution of workloads is roughly centred around medium:

$$
\pi =
\begin{bmatrix}
0.10 & 0.20 & 0.30 & 0.25 & 0.15
\end{bmatrix}.
$$

The HMM is defined with probabilities:

$$
A =
\begin{bmatrix}
0.60 & 0.20 & 0.10 & 0.05 & 0.05 \\
0.15 & 0.55 & 0.15 & 0.10 & 0.05 \\
0.10 & 0.15 & 0.55 & 0.15 & 0.05 \\
0.05 & 0.10 & 0.20 & 0.55 & 0.10 \\
0.05 & 0.05 & 0.10 & 0.25 & 0.55
\end{bmatrix}
;
\quad
B =
\begin{bmatrix}
0.05 & 0.15 & 0.30 & 0.50 \\
0.05 & 0.25 & 0.45 & 0.25 \\
0.10 & 0.30 & 0.40 & 0.20 \\
0.20 & 0.40 & 0.25 & 0.15 \\
0.35 & 0.40 & 0.15 & 0.10
\end{bmatrix}.
$$

Note: Rows in A correspond to workload states $0, . . . , 4$ and columns to subsequent state. Rows in B correspond to workload states $0, . . . , 4$ and columns correspond to engagement {absent, low, medium, high}.

## What you must compute

Your goal is to compute the posterior distribution over the final workload state:

$$
p(X_T \mid y_{1:T}).
$$

a) Draw the Bayesian graphical model for $T = 4$ time steps (variables $X_1, . . . , X_4$ and $Y_1, . . . , Y_4$).

b) Write the full joint distribution $p(x_{1:T}, y_{1:T})$ for arbitrary $T$ and factorise it according to your model assumptions.

c) In the attached notebook implement a function that computes $p(X_T \mid y_{1:T})$ by brute-force marginalisation:

$$
\tilde{p}(x_T) := p(x_T, y_{1:T}) = \sum_{x_{1:T-1}} p(x_{1:T}, y_{1:T});
\quad
p(x_T \mid y_{1:T}) = \frac{\tilde{p}(x_T)}{\sum_{x_T} \tilde{p}(x_T)}.
$$

This should enumerate all hidden state sequences $(x_1, . . . , x_T)$ and accumulate the probability mass onto $x_T$.

d) Run your brute-force method for $T = 6$ (the provided sequence) and for $T = 8$ (extend the provided sequence however you like). Briefly describe: what is the time complexity of the brute-force method as a function of $T$ and $|X|$? Why does it scale so badly?

e) Starting from our belief

$$
b_t(x_t) := p(x_t \mid y_{1:t}) \propto p(x_t, y_{1:t}),
$$

and using your factorisation from Part A, show that the beliefs satisfy the recursion

$$
b_t(x_t) \propto p(y_t \mid x_t)
\sum_{x_{t-1}} p(x_t \mid x_{t-1}) b_{t-1}(x_{t-1}).
$$

Your derivation should clearly indicate where you used (i) the Markov property and (ii) the conditional independence of observations.

f) Implement a belief-update function that takes $b_{t-1}$ and returns $b_t$ using the recursion from Part C. Ensure your result is a valid probability distribution (nonnegative and sums to 1).

g) Use your update to recursively compute $p(X_T \mid y_{1:T})$ for the provided $T = 6$ sequence.

h) Time your efficient implementation for $T = \{8, 50\}$. Briefly describe: why is the filtering method so much faster than brute force?

## Problem 2

You are given noisy observations of a 1D function:

$$
y_i = f(x_i) + \varepsilon_i, \quad \varepsilon_i \sim \mathcal{N}(0, \sigma^2)
$$

We place a Gaussian process prior on $f(x)$:

$$
f(x) \sim \text{GP}(0, k(x, x'))
$$

with RBF kernel:

$$
k(x, x') = \sigma_f^2
\exp\left(
-\frac{(x - x')^2}{2\ell^2}
\right).
$$

a) Use the attached notebook to implement the RBF kernel, plot kernel matrices for different lengthscales (assume $\sigma_f = 1$), and sample functions from the GP prior. Describe how lengthscale affects sample smoothness. Note that samples from a multivariate Gaussian can be taken using the Cholesky decomposition $K = LL^T$:

$$
z \sim \mathcal{N}(0, I)
$$

$$
f = Lz \sim \mathcal{N}(0, K)
$$

b) Use the attached notebook to implement the posterior predictive mean and covariance using Cholesky. Visually inspect the posterior samples. Describe why this method is preferred to explicitly computing a matrix inverse.

c) Use the attached notebook to implement log marginal likelihood, perform a grid search over lengthscales, and plot log marginal likelihood vs lengthscale. What (range of) lengthscale fits best for this problem based on this search?

d) Implement a numerical optimisation method to search for the best marginal likelihood. What lengthscale do you find is optimal?

e) Bonus question: it can be shown that a single-hidden-layer neural network with a particular form:

$$
f(x) = \frac{1}{\sqrt{m}} \sum_{i=1}^{m} a_i \phi(w_i^T x + b_i)
$$

converges in distribution to a GP as $m \to \infty$, assuming IID Gaussian weights, $a_i \sim \mathcal{N}(0, \sigma_a^2)$, $w_i \sim \mathcal{N}(0, \sigma_w^2 I)$, $b_i \sim \mathcal{N}(0, \sigma_b^2)$. Note that this converges to some GP, whose kernel depends on the activation $\phi$ (e.g., ReLU, tanh). Using the tanh activation for a smooth kernel function, create a sampler that samples neural networks using these assumptions for varying widths $m$. Describe what happens. At what point do you begin to see behaviour similar to the GP prior distribution?